<a href="https://colab.research.google.com/github/paulyu8868/test/blob/main/%EA%B0%9C%EB%B3%84%EB%A7%A4%EB%8F%84%EC%BC%88%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from dataclasses import dataclass

@dataclass
class Trade:
    trade_id: int
    buy_date: str
    buy_type: str
    buy_price: float
    buy_quantity: int
    buy_amount: float
    sell_date: str = None
    sell_type: str = None
    sell_price: float = None
    sell_amount: float = None
    holding_days: int = None
    returns: float = None

def round_half_up_to_two(num):
    try:
        if isinstance(num, (float, int)):
            num_100 = num * 100
            if num_100 - int(num_100) >= 0.5:
                return (int(num_100) + 1) / 100
            else:
                return int(num_100) / 100
        else:
            return num
    except:
        return num

def get_data(ticker, start, end):
    df = yf.download(ticker, start=start, end=end)
    df = df.drop(columns=['Volume', 'Adj Close'])
    df.index = df.index.date

    # 호가 단위 0.01$ 적용
    for col in ['Open', 'High', 'Low', 'Close']:
        df[col] = df[col].map(round_half_up_to_two)

    # 등락율 계산
    df['Return'] = df['Close'].pct_change() * 100
    df['Return'] = df['Return'].map(round_half_up_to_two)

    return df

def get_probability_from_updays(up_days):
    probability_table = {
        0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0,
        7: 0.0265,
        8: 0.1122,
        9: 0.3026,
        10: 0.4880,
        11: 0.7352,
        12: 0.8538,
        13: 0.9164,
        14: 0.9766,
        15: 0.9963,
        16: 1.0, 17: 1.0, 18: 1.0, 19: 1.0, 20: 1.0
    }
    return probability_table.get(up_days, 0.0)

def calculate_win_probability(df, current_idx):
    lookback_data = df.iloc[max(0, current_idx-20):current_idx]
    lookback_size = len(lookback_data)

    if lookback_size == 0:
        return 0.5

    up_days = (lookback_data['Return'].values > 0).sum()
    win_prob = get_probability_from_updays(up_days)

    return win_prob

def kelly_formula(win_prob, win_loss_ratio=1.0):
    q = 1 - win_prob
    f = (win_prob * win_loss_ratio - q) / win_loss_ratio
    return max(0.6, f)  # 0.6~1 사이로 제한

In [2]:
def infinite_buy_simulation(df, df_res, initial_funds, buy_portion, start_idx, simulation_period):
    funds = initial_funds
    one_buy_amount = initial_funds / buy_portion
    holdings = 0
    buy_records = []  # 각 매수 건을 저장할 리스트
    trade_history = []  # 완료된 매매 기록을 저장할 리스트
    trade_id = 1  # 매매 회차 ID

    df_res = pd.DataFrame(index=range(start_idx, start_idx + simulation_period + 1),
                         columns=['날짜', '시가', '고가', '종가', '등락율', '상승횟수', '승률', '켈리비율',
                                'LOC 매수', '수익 실현 매도', 'MOC 손절',
                                '보유 주식 수', '예수금', '총 평가액', '수익율(%)'])

    for i in range(start_idx, start_idx + simulation_period + 1):
        # 데이터 시리즈로 가져오기
        current_date = df.index[i]
        open_price = float(df['Open'].iloc[i])
        high_price = float(df['High'].iloc[i])
        close_price = float(df['Close'].iloc[i])
        return_val = float(df['Return'].iloc[i])
        prev_price = float(df['Close'].iloc[i-1]) if i > 0 else close_price

        # 기본 데이터 저장
        df_res.at[i, '날짜'] = current_date
        df_res.at[i, '시가'] = open_price
        df_res.at[i, '고가'] = high_price
        df_res.at[i, '종가'] = close_price
        df_res.at[i, '등락율'] = f"{round_half_up_to_two(return_val)}%"

        # 상승횟수, 승률, 켈리비율 계산
        lookback_data = df.iloc[max(0, i-20):i]
        up_days = (lookback_data['Return'].values > 0).sum()
        win_prob = get_probability_from_updays(up_days)
        kelly_ratio = kelly_formula(win_prob)

        df_res.at[i, '상승횟수'] = up_days
        df_res.at[i, '승률'] = f"{round_half_up_to_two(win_prob * 100)}%"
        df_res.at[i, '켈리비율'] = f"{round_half_up_to_two(kelly_ratio * 100)}%"

        price = close_price
        optimal_amount = one_buy_amount * kelly_ratio

        # 매수/매도 칼럼 초기화
        df_res.at[i, 'LOC 매수'] = 0
        df_res.at[i, '수익 실현 매도'] = 0
        df_res.at[i, 'MOC 손절'] = 0

        # 매수 로직
        if holdings == 0:
            # 첫 매수는 전날 종가의 1.15배에 LOC 매수
            loc_price = prev_price * 1.15
            if price <= loc_price:
                qty = int(optimal_amount / price)
                if funds >= qty * price:
                    holdings += qty
                    funds -= qty * price
                    buy_records.append({
                        'id': trade_id,
                        'buy_date': current_date,
                        'buy_price': price,
                        'quantity': qty,
                        'days': 0,
                        'type': 'LOC 매수'
                    })
                    df_res.at[i, 'LOC 매수'] = qty
                    trade_id += 1
        else:
            # 추가 매수는 전일 종가 대비 하락할 때만
            if price < prev_price:
                qty = int(optimal_amount / price)
                if funds >= qty * price:
                    holdings += qty
                    funds -= qty * price
                    buy_records.append({
                        'id': trade_id,
                        'buy_date': current_date,
                        'buy_price': price,
                        'quantity': qty,
                        'days': 0,
                        'type': 'LOC 매수'
                    })
                    df_res.at[i, 'LOC 매수'] = qty
                    trade_id += 1

        # 매도 로직
        if holdings > 0:
            new_buy_records = []  # 매도되지 않은 매수 건을 저장할 새 리스트
            total_sell = 0  # 당일 총 매도 수량

            for record in buy_records:
                record['days'] += 1  # 보유 일수 증가
                sell_price = record['buy_price'] * 1.0018  # 각 매수 건별 매도 목표가

                if price >= sell_price:  # 수익 실현 매도 조건 충족
                    funds += record['quantity'] * price
                    total_sell += record['quantity']
                    holdings -= record['quantity']
                    # 거래 기록 저장
                    trade_history.append({
                        '회차': record['id'],
                        '매수일': record['buy_date'],
                        '매수가': record['buy_price'],
                        '매수수량': record['quantity'],
                        '매도일': current_date,
                        '매도가': price,
                        '매도수량': record['quantity'],
                        '보유기간': record['days'],
                        '수익률(%)': round_half_up_to_two((price/record['buy_price'] - 1) * 100)
                    })
                elif record['days'] >= 30:  # 30일 경과 시 손절
                    funds += record['quantity'] * price
                    df_res.at[i, 'MOC 손절'] = record['quantity']
                    holdings -= record['quantity']
                    # 손절 거래 기록 저장
                    trade_history.append({
                        '회차': record['id'],
                        '매수일': record['buy_date'],
                        '매수가': record['buy_price'],
                        '매수수량': record['quantity'],
                        '매도일': current_date,
                        '매도가': price,
                        '매도수량': record['quantity'],
                        '보유기간': record['days'],
                        '수익률(%)': round_half_up_to_two((price/record['buy_price'] - 1) * 100)
                    })
                else:
                    new_buy_records.append(record)

            if total_sell > 0:
                df_res.at[i, '수익 실현 매도'] = total_sell

            buy_records = new_buy_records

        # 포트폴리오 상태 저장
        df_res.at[i, '보유 주식 수'] = holdings
        df_res.at[i, '예수금'] = round_half_up_to_two(funds)
        df_res.at[i, '총 평가액'] = round_half_up_to_two(funds + (price * holdings))
        df_res.at[i, '수익율(%)'] = round_half_up_to_two((funds + (price * holdings)) / initial_funds - 1) * 100

    final_value = funds + (holdings * float(df['Close'].iloc[start_idx + simulation_period]))
    return round_half_up_to_two((final_value / initial_funds - 1) * 100), df_res, final_value, pd.DataFrame(trade_history)

In [7]:
if __name__ == "__main__":
    start_date = '2022-01-01'
    end_date = '2023-01-31'
    initial_funds = 40000
    buy_portion = 6  # 6분할

    # 시작일 30일 전의 날짜
    start_date_dt = datetime.strptime(start_date,'%Y-%m-%d')
    start_date_before_30 = start_date_dt - timedelta(days=30)
    start_date_before_30 = start_date_before_30.strftime('%Y-%m-%d')

    # 종료일 다음 날 계산
    end_date_dt = datetime.strptime(end_date, '%Y-%m-%d')
    next_day = end_date_dt + timedelta(days=1)
    end_day_next = next_day.strftime('%Y-%m-%d')

    # 30일 전 데이터부터 입력
    df = get_data(ticker='TQQQ', start=start_date_before_30, end=end_day_next)
    df_length = len(df) - len(get_data(ticker='TQQQ', start=start_date, end=end_day_next))

    df_res = pd.DataFrame(columns=['날짜', '시가', '고가', '종가', '등락율', 'LOC 매수', '수익 실현 매도', 'MOC 손절',
                                  '보유 주식 수', '예수금', '총 평가액', '수익율(%)'])

    # 시뮬레이션 실행
    return_rate, df_res, final_value, df_trades = infinite_buy_simulation(
        df, df_res, initial_funds, buy_portion, df_length, len(df)-1-df_length)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
<ipython-input-2-20b6c6fb1404>:17: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  open_price = float(df['Open'].iloc[i])
<ipython-input-2-20b6c6fb1404>:18: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  high_price = float(df['High'].iloc[i])
<ipython-input-2-20b6c6fb1404>:19: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  close_price = float(df['Close'].iloc[i])
<ipython-input-2-20b6c6fb1404>:21: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  prev_price = float(df['

In [8]:
# 일별 거래 현황 출력
print("\n[일별 거래 현황]")
df_res_style = df_res.style.format({
    '시가': '{:.2f}',
    '고가': '{:.2f}',
    '종가': '{:.2f}',
    'LOC 매수': '{:.0f}',
    '수익 실현 매도': '{:.0f}',
    'MOC 손절': '{:.0f}',
    '보유 주식 수': '{:.0f}',
    '예수금': '{:.2f}',
    '총 평가액': '{:.2f}'
}).set_properties(**{'text-align': 'center'})
display(df_res_style)


[일별 거래 현황]


,날짜,시가,고가,종가,등락율,상승횟수,승률,켈리비율,LOC 매수,수익 실현 매도,MOC 손절,보유 주식 수,예수금,총 평가액,수익율(%)
21,2022-01-03,83.92,85.72,85.57,2.89%,9,30.26%,60.0%,46,0,0,46,36063.78,40000.00,0.000000
22,2022-01-04,85.92,85.94,82.24,-3.89%,10,48.8%,60.0%,48,0,0,94,32116.26,39846.82,0.000000
23,2022-01-05,81.19,81.86,74.68,-9.19%,9,30.26%,60.0%,53,0,0,147,28158.22,39136.18,-2.000000
24,2022-01-06,73.51,76.40,74.47,-0.28%,8,11.22%,60.0%,53,0,0,200,24211.31,39105.31,-2.000000
25,2022-01-07,74.45,75.47,72.04,-3.26%,7,2.65%,60.0%,55,0,0,255,20249.11,38619.31,-3.000000
26,2022-01-10,69.18,72.47,72.30,0.36%,7,2.65%,60.0%,0,55,0,200,24225.61,38685.61,-3.000000
27,2022-01-11,71.71,75.51,75.48,4.4%,7,2.65%,60.0%,0,106,0,94,32226.49,39321.61,-1.000000
28,2022-01-12,76.99,77.96,76.34,1.14%,8,11.22%,60.0%,0,0,0,94,32226.49,39402.45,-1.000000
29,2022-01-13,77.13,77.55,70.80,-7.25%,9,30.26%,60.0%,56,0,0,150,28261.69,38881.69,-2.000000
30,2022-01-14,69.09,72.07,71.87,1.51%,8,11.22%,60.0%,0,56,0,94,32286.41,39042.19,-2.000000


In [9]:
# 매매 기록 출력
if not df_trades.empty:
    print("\n[매매 기록]")
    df_trades_style = df_trades.style.format({
        '매수가': '{:.2f}',
        '매수수량': '{:.0f}',
        '매도가': '{:.2f}',
        '매도수량': '{:.0f}',
        '수익률(%)': '{:.2f}'
    }).set_properties(**{'text-align': 'center'})

    # 수익률에 따른 색상 적용
    def color_returns(val):
        color = 'red' if val < 0 else ('blue' if val > 0 else 'black')
        return f'color: {color}'

    df_trades_style = df_trades_style.applymap(color_returns, subset=['수익률(%)'])
    display(df_trades_style)

    # 매매 통계 출력
    print('\n' + '='*80)
    print("매매 통계")
    print('='*80)
    print(f"총 매매 횟수: {len(df_trades)} 회")
    print(f"평균 보유기간: {df_trades['보유기간'].mean():.1f} 일")
    print(f"평균 수익률: {df_trades['수익률(%)'].mean():.2f}%")
    win_rate = len(df_trades[df_trades['수익률(%)'] > 0]) / len(df_trades) * 100
    print(f"승률: {win_rate:.2f}%")

    print('\n' + '='*80)
    print(f"{start_date} ~ {end_date} 동안의 자산 변동 결과")
    print('='*80)
    print(f"최초 보유 금액: ${initial_funds:,.2f}")
    print(f"최종 보유 금액: ${final_value:,.2f}")
    print(f"원금 변화율: {return_rate}%")
    print('='*80)


[매매 기록]


<ipython-input-9-606dc99f0915>:17: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  df_trades_style = df_trades_style.applymap(color_returns, subset=['수익률(%)'])


,회차,매수일,매수가,매수수량,매도일,매도가,매도수량,보유기간,수익률(%)
0,5,2022-01-07,72.04,55,2022-01-10,72.30,55,2,0.36
1,3,2022-01-05,74.68,53,2022-01-11,75.48,53,5,1.07
2,4,2022-01-06,74.47,53,2022-01-11,75.48,53,4,1.36
3,6,2022-01-13,70.80,56,2022-01-14,71.87,56,2,1.51
4,10,2022-01-21,56.67,70,2022-01-24,57.49,70,2,1.45
5,11,2022-01-25,53.25,75,2022-01-28,56.37,75,4,5.86
6,12,2022-01-26,53.15,75,2022-01-28,56.37,75,3,6.06
7,13,2022-01-27,51.56,77,2022-01-28,56.37,77,2,9.33
8,9,2022-01-20,61.88,64,2022-02-01,63.01,64,9,1.83
9,8,2022-01-19,64.37,62,2022-02-02,64.50,62,11,0.20



매매 통계
총 매매 횟수: 154 회
평균 보유기간: 7.4 일
평균 수익률: 0.41%
승률: 88.31%

2022-01-01 ~ 2023-01-31 동안의 자산 변동 결과
최초 보유 금액: $40,000.00
최종 보유 금액: $43,083.29
원금 변화율: 7.71%


In [10]:
# 기간별 성과 분석
def analyze_performance_by_period(df_trades, period='M'):
    """
    period options:
    - 'M': 월별
    - 'Q': 분기별
    - 'Y': 연도별
    """
    # 매수일을 datetime으로 변환
    df_trades['매수일'] = pd.to_datetime(df_trades['매수일'])

    # 기간별 그룹화
    df_trades['period'] = df_trades['매수일'].dt.to_period(period)

    # 기간별 성과 계산
    performance = []
    for period, group in df_trades.groupby('period'):
        profit_trades = group[group['수익률(%)'] > 0]
        loss_trades = group[group['수익률(%)'] <= 0]

        performance.append({
            '기간': period.strftime('%Y-%m'),
            '거래횟수': len(group),
            '승률(%)': round(len(profit_trades) / len(group) * 100, 2),
            '평균수익률(%)': round(profit_trades['수익률(%)'].mean() if len(profit_trades) > 0 else 0, 2),
            '평균손실률(%)': round(loss_trades['수익률(%)'].mean() if len(loss_trades) > 0 else 0, 2),
            '전체평균수익률(%)': round(group['수익률(%)'].mean(), 2),
            '최대수익률(%)': round(group['수익률(%)'].max(), 2),
            '최대손실률(%)': round(group['수익률(%)'].min(), 2),
            '평균보유기간': round(group['보유기간'].mean(), 1)
        })

    df_performance = pd.DataFrame(performance)

    # 스타일링
    def style_percentage(val):
        try:
            num = float(str(val).replace('%', ''))
            color = 'red' if num < 0 else ('blue' if num > 0 else 'black')
            return f'color: {color}'
        except:
            return 'color: black'

    df_performance_style = df_performance.style.format({
        '승률(%)': '{:.2f}',
        '평균수익률(%)': '{:.2f}',
        '평균손실률(%)': '{:.2f}',
        '전체평균수익률(%)': '{:.2f}',
        '최대수익률(%)': '{:.2f}',
        '최대손실률(%)': '{:.2f}',
        '평균보유기간': '{:.1f}'
    }).set_properties(**{
        'text-align': 'center',
        'font-family': 'NanumGothic',
        'width': '100px'
    }).applymap(style_percentage, subset=[
        '평균수익률(%)',
        '평균손실률(%)',
        '전체평균수익률(%)',
        '최대수익률(%)',
        '최대손실률(%)'
    ])

    return df_performance_style

# 기간별 분석 출력 (월별)
print("\n[월별 성과 분석]")
monthly_performance = analyze_performance_by_period(df_trades, 'M')
display(monthly_performance)

# 기간별 분석 출력 (분기별)
print("\n[분기별 성과 분석]")
quarterly_performance = analyze_performance_by_period(df_trades, 'Q')
display(quarterly_performance)

# 기간별 분석 출력 (연도별)
print("\n[연도별 성과 분석]")
yearly_performance = analyze_performance_by_period(df_trades, 'Y')
display(yearly_performance)

# 전체 통계 요약
print("\n[전체 기간 통계 요약]")
print("="*80)
profit_trades = df_trades[df_trades['수익률(%)'] > 0]
loss_trades = df_trades[df_trades['수익률(%)'] <= 0]

print(f"총 거래 횟수: {len(df_trades)} 회")
print(f"전체 승률: {len(profit_trades)/len(df_trades)*100:.2f}%")
print(f"전체 평균 수익률: {df_trades['수익률(%)'].mean():.2f}%")
print(f"평균 수익 (이익 거래): {profit_trades['수익률(%)'].mean():.2f}%")
print(f"평균 손실 (손실 거래): {loss_trades['수익률(%)'].mean():.2f}%")
print(f"최대 수익률: {df_trades['수익률(%)'].max():.2f}%")
print(f"최대 손실률: {df_trades['수익률(%)'].min():.2f}%")
print(f"평균 보유기간: {df_trades['보유기간'].mean():.1f}일")
print(f"수익 대 손실 비율: {abs(profit_trades['수익률(%)'].mean()/loss_trades['수익률(%)'].mean()):.2f}")
print("="*80)


[월별 성과 분석]


<ipython-input-10-6296c4906d20>:44: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  df_performance_style = df_performance.style.format({


,기간,거래횟수,승률(%),평균수익률(%),평균손실률(%),전체평균수익률(%),최대수익률(%),최대손실률(%),평균보유기간
0,2022-01,13,76.92,2.90,-30.94,-4.91,9.33,-37.55,10.3
1,2022-02,9,88.89,3.20,-3.06,2.50,9.79,-3.06,9.0
2,2022-03,14,92.86,5.75,-49.54,1.80,10.70,-49.54,5.8
3,2022-04,13,69.23,5.21,-36.28,-7.56,10.57,-47.26,10.8
4,2022-05,11,90.91,5.32,-39.04,1.28,10.87,-39.04,6.5
5,2022-06,11,90.91,2.98,-4.56,2.29,8.12,-4.56,8.3
6,2022-07,8,100.00,5.06,0.00,5.06,12.48,0.39,2.9
7,2022-08,17,82.35,3.57,-45.61,-5.11,8.21,-45.95,8.9
8,2022-09,12,83.33,3.40,-13.73,0.55,6.94,-14.80,7.8
9,2022-10,11,100.00,5.08,0.00,5.08,9.85,1.44,4.7



[분기별 성과 분석]


<ipython-input-10-6296c4906d20>:44: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  df_performance_style = df_performance.style.format({


,기간,거래횟수,승률(%),평균수익률(%),평균손실률(%),전체평균수익률(%),최대수익률(%),최대손실률(%),평균보유기간
0,2022-03,36,86.11,4.17,-29.09,-0.44,10.70,-49.54,8.2
1,2022-06,35,82.86,4.48,-31.45,-1.68,10.87,-47.26,8.7
2,2022-09,37,86.49,3.89,-32.86,-1.08,12.48,-45.95,7.2
3,2022-12,39,94.87,4.59,-14.40,3.61,21.87,-15.15,6.5
4,2023-03,7,100.00,5.37,0.00,5.37,8.17,1.54,2.3



[연도별 성과 분석]


<ipython-input-10-6296c4906d20>:44: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  df_performance_style = df_performance.style.format({


,기간,거래횟수,승률(%),평균수익률(%),평균손실률(%),전체평균수익률(%),최대수익률(%),최대손실률(%),평균보유기간
0,2022-12,147,87.76,4.29,-29.29,0.18,21.87,-49.54,7.6
1,2023-12,7,100.00,5.37,0.00,5.37,8.17,1.54,2.3



[전체 기간 통계 요약]
총 거래 횟수: 154 회
전체 승률: 88.31%
전체 평균 수익률: 0.41%
평균 수익 (이익 거래): 4.35%
평균 손실 (손실 거래): -29.29%
최대 수익률: 21.87%
최대 손실률: -49.54%
평균 보유기간: 7.4일
수익 대 손실 비율: 0.15
